# Libraries Used

Pandas: for data manipulation and file handling
os: for working with file paths

In [ ]:
import os
import pandas as pd
from IPython.display import display 
from datetime import datetime

# Processing Phase Overview

This notebook documents the complete data processing workflow applied to the prepared datasets.

#### Initial Validation and Exploration

Before applying any transformations, an initial data exploration was conducted using the custom function `analyze_all_csv()`.  
The results of this exploration were saved in dedicated validation notebooks located in the `validation` folder — one for each processing stage.

These validation steps helped identify several key issues, including:

- Missing values  
- Duplicate records  
- Outliers in numeric columns  
- Inconsistent data types (e.g., date columns stored as strings)  
- Columns with irrelevant or unreliable data

#### Data Cleaning Actions Based on Validation Insights

The insights gathered from the validation reports directly informed the following cleaning steps:

- Standardizing date and time formats  
- Changing data types for consistency  
- Removing invalid or extreme outliers  
- Dropping nulls and duplicate rows  
- Eliminating columns with unusable values (e.g., the `Fat` column in the weight log)

#### Note

Some transformation steps — such as wide-to-long restructuring, concatenation, and early duplicate removal — were already applied during the **Preparing Phase**.  
These early actions were taken to reduce redundancy, optimize performance, and prepare the data structure for downstream tasks.

#### Assurance of Quality and Transparency

Each decision in this notebook is supported by evidence from the validation phase.  
This structured approach ensures transparency, reproducibility, and integrity throughout the entire data preparation process.


# Standardizing Datetime Columns Across Datasets

To ensure consistency across all datasets, all date-related columns were standardized using a robust multi-format datetime parser.  
This parser handled inconsistently formatted strings such as:

- `"4/13/2016 00:00"`
- `"04/13/2016 12:00 AM"`
- `"2016-04-13 00:00:00"`  
  etc.

Each value was parsed into a proper datetime object and then exported to a unified string format:  
**`"YYYY-MM-DD HH:MM:SS"`**

## Why this step was important:

-  **Enabled accurate time-based operations**  
  (e.g., sorting, grouping, time-series analysis)

-  **Eliminated format inconsistencies**  
  across merged datasets

-  **Allowed hidden duplicates**  
  (same datetime but different formats) to be revealed and addressed

-  **Prevented downstream parsing issues**  
  by enforcing a reliable and consistent structure

##  Technical Note: Datatype vs CSV Limitations

Although columns were temporarily converted to `datetime64[ns]` (Pandas' native datetime type), **CSV files do not preserve data types**.  
As a result, these columns are saved as plain strings (`object`) in the exported CSVs.

>  These datetime columns will be **re-parsed as datetime during the analysis phase** as needed.


In [ ]:

# Define folder paths
input_folder = 'final_prepared_data'
output_folder = 'data_after_standardization_and_type_change'

# Robust datetime parser with multiple format fallbacks
def try_parse_date(date_str):
    formats = [
        "%Y-%m-%d", "%m/%d/%Y", "%m/%d/%Y %I:%M:%S %p", "%m/%d/%Y %H:%M:%S",
        "%m/%d/%Y %I:%M %p", "%Y-%m-%d %H:%M:%S", "%d/%m/%Y",
        "%m/%d/%Y %I:%M:%S", "%m/%d/%Y %H:%M"
    ]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return pd.NaT

# Files and their datetime columns
files = [
    ('prepared_activity_daily_full_appended.csv', 'ActivityDate'),
    ('prepared_calories_hourly_appended.csv', 'ActivityHour'),
    ('prepared_calories_per_minute.csv', 'ActivityMinute'),
    ('prepared_heartrate_per_second_appended.csv', 'Time'),
    ('prepared_intensity_hourly_appended.csv', 'ActivityHour'),
    ('prepared_intensity_per_minute.csv', 'ActivityMinute'),
    ('prepared_mets_per_minute_narrow_appended.csv', 'ActivityMinute'),
    ('prepared_sleep_per_minute_appended.csv', 'date'),
    ('prepared_sleep_summary_daily_2016_v01.csv', 'SleepDay'),
    ('prepared_steps_hourly_appended.csv', 'ActivityHour'),
    ('prepared_steps_per_minute.csv', 'ActivityMinute'),
    ('prepared_weight_log_full_appended.csv', 'Date'),
]

# Unified string format (date + optional time)
def format_date(dt):
    return dt.strftime("%Y-%m-%d %H:%M:%S") if pd.notna(dt) else ""

# Process files
for filename, column in files:
    df = pd.read_csv(os.path.join(input_folder, filename))
    df[column] = df[column].apply(try_parse_date).apply(format_date)
    df.to_csv(os.path.join(output_folder, filename), index=False)
    print(f"[✓] Saved: {filename}")


### Handling Missing Values

After inspecting missing values across all datasets using the `isnull().sum()` and percentage calculation functions, I found that the **`Fat` column** in the `prepared_weight_log_full_appended.csv` file contains **94 missing values**, representing **95.92%** of the total entries.

Due to the extremely high proportion of missing data in this column (over 95%), and the fact that no other columns across the datasets contain missing values, the `Fat` column was **dropped** from the dataset as it would not contribute meaningfully to the analysis.

This decision aligns with standard data cleaning practices, where columns with excessive missing data are considered unreliable for statistical analysis.


In [ ]:

# === Load the file ===
df = pd.read_csv('data_after_standardization_and_type_change/prepared_weight_log_full_appended.csv')

# === Drop the 'Fat' column ===
df.drop(columns=['Fat'], inplace=True)

# === Save the cleaned version ===
output_folder = 'data_after_removing_nulls_and_duplicates'

output_path = os.path.join(output_folder,'prepared_weight_log_full_appendedv2.csv' )

df.to_csv(output_path, index=False)

print(f"Cleaned file saved")


# Handling Duplicate Records

To maintain data integrity and avoid inflated row counts, duplicate records were removed .

During the earlier **Prepare** phase, duplicates had already been removed from some concatenated files using `drop_duplicates()`.  
This helped reduce file size and improve processing performance.

However, after applying consistent datetime formatting in the **Standardization** phase, previously hidden duplicates became detectable.

Many datetime columns had been originally stored using inconsistent formats, such as:

- `4/13/2016 00:00`  
- `04/13/2016 12:00 AM`  
- `2016-04-13 00:00:00`

While these values represent the same timestamp, their formatting differences caused them to appear unique.  
After transforming all datetime values to a unified format (`YYYY-MM-DD HH:MM:SS`), the system was able to correctly identify and remove them as duplicates.

This led to an increase in duplicate rows in the following files:

- `prepared_calories_per_minute.csv`  
- `prepared_intensity_per_minute.csv`  
- `prepared_steps_per_minute.csv`

Removing these duplicates was essential to maintaining accuracy in time-based metrics and ensuring the reliability of analysis results.

The file `prepared_sleep_summary_daily_2016_v01.csv` also contained three residual duplicate rows, which were removed to ensure full consistency.





In [ ]:
# === Folder paths ===
input_folder = 'data_after_standardization_and_type_change'
output_folder = 'data_after_removing_nulls_and_duplicates'
os.makedirs(output_folder, exist_ok=True)

# === Files to process ===
files_to_deduplicate = [
    'prepared_calories_per_minute.csv',
    'prepared_intensity_per_minute.csv',
    'prepared_steps_per_minute.csv',
    'prepared_sleep_summary_daily_2016_v01.csv',
]

# === Deduplication loop ===
for filename in files_to_deduplicate:
    file_path = os.path.join(input_folder, filename)
    df = pd.read_csv(file_path)

    original_rows = len(df)
    df = df.drop_duplicates()
    removed = original_rows - len(df)

    # Save the cleaned file
    output_path = os.path.join(output_folder, filename)
    df.to_csv(output_path, index=False)

    print(f"[✓] Processed {filename}")
    print(f"    → Original rows: {original_rows}")
    print(f"    → Duplicates removed: {removed}")
    print(f"    → Final rows: {len(df)}\n")


# Handling Outliers Using the IQR Method

To enhance the quality and reliability of the data, outlier removal was applied to all numeric columns (excluding identifier columns) using the Interquartile Range (IQR) method.

## Methodology

For each numeric column in each CSV file:

- The interquartile range (IQR) was calculated as:  
  `IQR = Q3 - Q1`

- Outliers were defined as values outside the following range:
  - `Lower bound = Q1 - 1.5 * IQR`
  - `Upper bound = Q3 + 1.5 * IQR`

- If more than **10%** of the values in a column were identified as outliers, the column was skipped to avoid excessive data loss.

## Additional Cleaning Rules

- All negative numeric values (i.e., values less than 0) were removed from the dataset, since such values are not expected in this context  
  (e.g., negative steps, calories, or active minutes).

## Columns Excluded from Outlier Detection

- The `Id` column was excluded from all outlier detection processes in order to preserve the integrity of user identifiers.



In [ ]:
import os
import pandas as pd

# === Folder paths ===
input_folder = 'data_after_removing_nulls_and_duplicates'
output_folder = 'data_after_removing_outliers'
os.makedirs(output_folder, exist_ok=True)

# === Function to remove outliers using IQR ===
def remove_outliers_iqr(df, exclude_cols=None, outlier_threshold=10.0):
    if exclude_cols is None:
        exclude_cols = []

    numeric_cols = df.select_dtypes(include=["number"]).columns
    for col in numeric_cols:
        if col in exclude_cols:
            continue

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Count outliers
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_percent = (len(outliers) / len(df)) * 100

        # Remove outliers only if % is ≤ threshold
        if outlier_percent <= outlier_threshold:
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

# === Process all CSV files ===
for filename in os.listdir(input_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_folder, filename)
        df = pd.read_csv(file_path)

        # Drop outliers from numeric columns (except 'Id') and values < 0
        df = remove_outliers_iqr(df, exclude_cols=['Id'])

        # Remove any numeric values < 0
        for col in df.select_dtypes(include=["number"]).columns:
            df = df[df[col] >= 0]

        # Save cleaned file
        output_path = os.path.join(output_folder, filename)
        df.to_csv(output_path, index=False)
